# DSCI552: HW1
### Alain Tamazian
### ID#: 2073746513

-------------
### Important Notice:

Side note 1:
I figured out that using the ".dat" instead of ".arff" can output slightly different results since ".dat" uses rounded values. Such as difference is very noticable in d.ii. It also leads to a small discepency in d.i.A, and possibly in other questions too. Please keep that in mind when grading.

Side note 2:
I was told by TAs that 1 is not a valid k* since it uses itself as the nearest neighbors. Accordingly, I tweaked my program to never get k* = 1. Otherwise, k* = 1 for d.i.A and d.ii, which would by extension affect the output of d.i.B. Please keep that in mind.

------------------
## Part (a) and (b)

In [1]:
# Cell for all imports necessary for HW1

from scipy.io.arff import loadarff 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.metrics import confusion_matrix
from warnings import simplefilter

In [2]:
file = "../data/vertebral_column_data/column_2C_weka.arff"

med_data = loadarff(file)
med_df = pd.DataFrame(med_data[0])

pd.set_option('display.max_rows', 10)
# uncomment the line below if you want all rows of data displayed instead of just a sample of 10
# pd.set_option('display.max_rows', None)

# display(med_df)

# Could alternatively use the .dat file, by uncommenting the 3 lines below
# file = "../data/vertebral_column_data/column_2C.dat"
# col_names = ["pelvic_incidence", "pelvic_tilt", "lumbar_lordosis_angle", 
#          "sacral_slope", "pelvic_radius", "grade_of_spondylolisthesis", "class"]
# med_df = pd.read_csv(file, header = None, sep = "\s+", names = col_names)

In [3]:
# We need to convert labels to 0 and 1 first
# I am doing so by converting the dataframe to a dictionary 

med_dict = {}

for col in med_df.head():
    med_dict[col] = []

for index, row in med_df.iterrows():
    for col in med_dict.keys():
        med_dict[col].append(row[col])
    
# print(med_dict)

In [4]:
labels = []

for label in med_dict["class"]:
    label = str(label)
    # I added the "or" operator in case you do run the program with the .dat file instead
    if label == "b'Abnormal'" or label == "AB":
        labels.append(1)
    else:
        labels.append(0)

# resets and paris the existing "class" key with the new binary label values we converted to
med_dict["class"] = labels
# print(med_dict)

# This is our new dataframe with the classes encoded with 1 and 0
med_df = pd.DataFrame(med_dict)
display(med_df)

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.027817,22.552586,39.609117,40.475232,98.672917,-0.254400,1
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,1
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,1
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,1
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,1
...,...,...,...,...,...,...,...
305,47.903565,13.616688,36.000000,34.286877,117.449062,-4.245395,0
306,53.936748,20.721496,29.220534,33.215251,114.365845,-0.421010,0
307,61.446597,22.694968,46.170347,38.751628,125.670725,-2.707880,0
308,45.252792,8.693157,41.583126,36.559635,118.545842,0.214750,0


-----------
## (b).i. 
Pre-Processing and Exploratory data analysis:

i. Make scatterplots of the independent variables in the dataset. Use color to show Classes 0 and 1.

In [ ]:
# using the seaborn.pairplot() function to make the scatterplots
# https://seaborn.pydata.org/generated/seaborn.pairplot.html

pairplot = sns.pairplot(med_df, hue = "class")

# Blue dots are obervations with a 0 (normal), and orange for 1 (abnormal)
plt.show()


----------------------
## (b).ii.
Pre-Processing and Exploratory data analysis:

ii. Make boxplots for each of the independent variables. Use color to show Classes 0 and 1 (see ISLR p. 129).

In [ ]:
# using the seaborn.boxplot() function
# https://seaborn.pydata.org/generated/seaborn.boxplot.html

# doing this to loop through all the independent variables 
# The last column is the dependent "class" variable so we exclude it
for col in list(med_dict.keys())[:-1]:
    boxplot = sns.boxplot(x="class", y=col, hue="class", data=med_df)
    plt.show()
    print()

----------------------
## (b).iii.
Pre-Processing and Exploratory data analysis:

iii. Select the first 70 rows of Class 0 and the first 140 rows of Class 1 as the training set and the rest of the data as the test set.

In [ ]:
# divide the entire dataset into seperate training and testing sets

# Seperate the rows with class 0 from 1
class0_df = med_df[(med_df["class"] == 0)]
class1_df = med_df[(med_df["class"] == 1)]

# Create the training set dataframe with the first 70 rows of Class 0 and the first 140 rows of Class 1
train_set = [class0_df[:70], class1_df[:140]]
# ignore_index argument basically, resets the index for the new df
train_set_df = pd.concat(train_set, ignore_index=True)
print("Training Set")
display(train_set_df)

# Create the testing set dataframe with the remaining observations 
test_set = [class0_df[70:], class1_df[140:]]
test_set_df = pd.concat(test_set, ignore_index=True)
print("\nTesting Set")
display(test_set_df)

----------------------
## (c).i.
Classification using KNN on Vertebral Column Data Set

i. Write code for k-nearest neighbors with Euclidean metric (or use a software package).

In [ ]:
# Will be using the scikit-learn package/libary
# specifically, the sklearn.neighborsKNeighborsClassifier algorithm

# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier



----------------------
## (c).ii.
Classification using KNN on Vertebral Column Data Set

ii. Test all the data in the test database with k nearest neighbors. Take decisions by majority polling. Plot train and test errors in terms of k for k ∈ {208, 205, . . . , 7, 4, 1, } (in reverse order). You are welcome to use smaller increments of k. Which k∗ is the most suitable k among those values? Calculate the confusion matrix, true positive rate, true negative rate, precision, and F1-score when k = k*.

In [ ]:
k_vals = list(range(208, 0, -3))

# Split training set dateframe into two dataframes -- one for the features (x) and one for the labels (y)
col_names = list(med_dict.keys())[:-1]
train_attr_df = train_set_df[col_names]
train_class_df = train_set_df["class"]
# Split testing set dateframe into two dataframes -- one for the features (x) and one for the labels (y)
test_attr_df = test_set_df[col_names]
test_class_df = test_set_df["class"]

# Initializing empty lists to gather the error rates associated with each k we test 
training_error_ratesL = []
testing_error_ratesL = []

sets = [(train_attr_df, train_class_df, training_error_ratesL), (test_attr_df, test_class_df, testing_error_ratesL)]

for attrs, labels, error_ratesL in sets:

    for k in k_vals:
        
        # specify the paramters of our model
        model = knn(n_neighbors=k, metric = "euclidean")
        # Alternatively, could set the power paramter p = 2 to get the Euclidean metric
        
        # We train/fit our model with the training set data
        model.fit(train_attr_df, train_class_df)
        
        errors = 0
        # using count to track the indices during the loop and compare the predicted class with the true ones
        count = 0
        # the predict() method classifies the unlabeled data based on the previous training of the model
        for label in model.predict(attrs):
            if label != labels[count]:
                errors += 1
            count += 1

        # error rate is the number of misclassified points divided by number of all test data
        error_rate = errors/len(model.predict(attrs))

        error_ratesL.append(error_rate)


In [ ]:
# Line graph of the training and testing error rate in terms of k for k ∈ {208, 205, . . . , 7, 4, 1, } (in reverse order)
plt.plot(k_vals, testing_error_ratesL)
plt.plot(k_vals, training_error_ratesL)
plt.xlabel("K")
plt.ylabel("Error Rate")
plt.legend(["Testing Error", "Training Error"])
plt.grid(True)
# By inverting the standard positions of the xlim() the graph is in reverse x order
# add +5 simply for visual reasons so the graph doesn't start and stop exactly with the plot line
plt.xlim(max(k_vals)+5, min(k_vals)-5)
plt.show()

# Not asked for, but helpful for interpretation
df = pd.DataFrame()
df["k"] = k_vals
df["Training Error Rate"] = training_error_ratesL
df["Testing Error Rate"] = testing_error_ratesL
display(df)


In [ ]:
# initializing best_k such that the while loop starts
best_k = 1
# Was told that 1 cannot be a valid k* since its using itself as a neighbor; need to exclude the possibility k* = 1
while best_k == 1:
    best_testing_error_rate = min(testing_error_ratesL)
    # using [::-1] since k_vals is sorted hgihest to lowest
    best_k = k_vals[::-1][testing_error_ratesL[::-1].index(best_testing_error_rate)]
    if 1 in k_vals: k_vals.remove(1)
# need to return the 1, since k_vals is often reused without being redefined
k_vals.insert(len(k_vals), 1)

print("For KNN Classification with Euclidean metric:")
print("k* =", best_k)
print("Resulting in best testing error rate of", best_testing_error_rate)
# classmate got 5; check with others

# Getting the best classifier and its results (at k*) -- to calculate measures accuracy
best_model = knn(n_neighbors=best_k, metric = "euclidean")
best_model.fit(train_attr_df, train_class_df)
best_pred = best_model.predict(test_attr_df)

# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
matrix = confusion_matrix(test_class_df, best_pred)

# https://seaborn.pydata.org/generated/seaborn.heatmap.html
sns.heatmap(matrix, annot=True)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

matrix_flat = matrix.flatten()
tn = matrix_flat[0]
fp = matrix_flat[1]
fn = matrix_flat[2]
tp = matrix_flat[3]
print("TN =", tn, "\nFP =", fp, "\nFN =", fn, "\nTP =", tp)

tpr = tp/(tp+fn)
tnr = tn/(tn+fp)
prec = tp/(tp+fp)
f1 = tp/(tp+0.5*(fp+fn))
print("TPR =", tpr)
print("TNR =", tnr)
print("Precision =", prec)
print("F1-score =", f1)

# Also, there is a function that gives the percision and f1-score; uncomment lines below if see
# from sklearn.metrics import classification_report
# print()
# report = classification_report(test_class_df, best_pred)
# print(report)

--------------------
## (c).iii 
Classification using KNN on Vertebral Column Data Set

iii. Since the computation time depends on the size of the training set, one may only use a subset of the training set. Plot the best test error rate, which is obtained by some value of k, against the size of training set, when the size of training set is N ∈ {10,20,30,...,210}. Note: for each N, select your training set by choosing the first ⌊N/3⌋ rows of Class 0 and the first N − ⌊N/3⌋ rows of Class 1 in the training set you created in 1(b)iii. Also, for each N, select the optimal k from a set starting from k = 1, increasing by 5. For example, if N = 200, the optimal k is selected from {1,6,11,...,196}. This plot is called a Learning Curve.

In [ ]:
# 1.c.iii

n_list = list(range(10, 211, 10))

# display(test_set_df)

best_ks_list = []
best_testing_error_ratesL = []

# looping through the list of n's to record the best k and test error rate based on training sample size
for n in n_list:

    train_set = [class0_df[:(n//3)], class1_df[:(n-n//3)]]
    # ignore_index argument basically, resets the index for the new df
    train_set_df = pd.concat(train_set, ignore_index=True)
    
    train_attr_df = train_set_df[col_names]
    train_class_df = train_set_df["class"]
    
    k_vals = list(range(1, n, 5))
    
    testing_error_ratesL = []

    for k in k_vals:
        
        # We build our model per our k and metric parameter specifications
        model = knn(n_neighbors=k, metric = "euclidean")
        # We train the model
        model.fit(train_attr_df, train_class_df)
        
        errors = 0
        # using count to track the indices during the loop and compare the predicted class with the true ones
        count = 0
        # the predict() method classifies the unlabeled data based on the previous training of the model
        for label in model.predict(test_attr_df):
            if label != test_class_df[count]:
                errors += 1
            count += 1

        # error rate is the number of misclassified points divided by number of all test data
        testing_error_rate = errors/len(model.predict(test_attr_df))
        testing_error_ratesL.append(testing_error_rate)
    
    # initializing best_k such that the while loop starts
    best_k = 1
    # Was told that 1 cannot be a valid k* since its using itself as a neighbor; need to exclude the possibility k* = 1
    while best_k == 1:
        # Even if there are multiple testing error rates of the same best size, since k_vals is sorted least to greatest we still get the optimal k by selecting the first applicable k
        best_testing_error_rate = min(testing_error_ratesL)
        best_k = k_vals[testing_error_ratesL.index(best_testing_error_rate)]
        if 1 in k_vals: k_vals.remove(1)
    # need to return the 1, since k_vals is often reused without being redefined
    k_vals.insert(0, 1)
    
    best_testing_error_ratesL.append(best_testing_error_rate)
    best_ks_list.append(best_k)
    

In [ ]:
# Line graph of the best test error rate against training set size (N) for N ∈ {10,20,30,...,210}
# ie plot of the learning curve
plt.plot(n_list, best_testing_error_ratesL)
plt.xlabel("Size of Training Set (N)")
plt.ylabel("Best Test Error Rate")
plt.title("Learning Curve")
plt.grid(True)
plt.show()

# This displays a dataframe for the various data relevant to the Learning Curve
# Not asked for, but helpful for interpretation
ln_df = pd.DataFrame()
ln_df["N Size"] = n_list
ln_df["Best Test Error Rate"] = best_testing_error_ratesL
ln_df["Optimal k"] = best_ks_list
display(ln_df)

-------------------
## (d).i.A.
Replace the Euclidean metric with the following metrics5 and test them. Sum- marize the test errors (i.e., when k = k∗) in a table. Use all of your training data and select the best k when {1,6,11,...,196}.

i. Minkowski Distance:

A. which becomes Manhattan Distance with p = 1.

In [ ]:
# The code is fairly similar to (c), could optimize the code by creating a function for it

# can make more efficient by changing the name of train_set_df in c.iii, so I don't have to redefine it
train_set = [class0_df[:70], class1_df[:140]]
train_set_df = pd.concat(train_set, ignore_index=True)

k_vals = list(range(1, 200, 5))

testing_errorsL = []
testing_error_ratesL = []


for k in k_vals:

    # Here is the main difference for (d).i.A; we set the metric as "manhattan"
    # Alternatively can use the parameter p=1 to represent the Manhattan distance 
    # We build our model per our k and metric parameter specifications        
    model = knn(n_neighbors=k, metric = "manhattan")
    # We train the model
    model.fit(train_attr_df, train_class_df)

    errors = 0
    count = 0
    # the predict() method classifies the unlabeled data based on the previous training of the model
    for label in model.predict(test_attr_df):
        if label != test_class_df[count]:
            errors += 1
        count += 1
    
    testing_errorsL.append(errors)
    testing_error_rate = errors/len(model.predict(test_attr_df))
    testing_error_ratesL.append(testing_error_rate)

# initializing best_k such that the while loop starts
best_k = 1
# Was told that 1 cannot be a valid k* since its using itself as a neighbor; need to exclude the possibility k* = 1
while best_k == 1:
    # Even if there are multiple testing error rates of the same best size, since k_vals is sorted least to greatest we still get the optimal k by selecting the first applicable k
    best_testing_error_rate = min(testing_error_ratesL)
    best_k = k_vals[testing_error_ratesL.index(best_testing_error_rate)]
    if 1 in k_vals: k_vals.remove(1)
# need to return the 1, since k_vals is often reused without being redefined
k_vals.insert(0, 1)

print("For KNN Classification with Manhattan Distance:")
print("k* =", best_k)
print("Resulting in best testing error rate of", best_testing_error_rate)

df = pd.DataFrame()
df["k"] = k_vals
df["Testing Error"] = testing_errorsL
df["Testing Error Rate"] = testing_error_ratesL
display(df)

# Not asked for, but helpful for interpretation
# Line graph of the testing error rate in terms of k for k ∈ {1,6,11,...,196} with Manhattan Distance
plt.plot(k_vals, testing_error_ratesL)
plt.xlabel("k")
plt.ylabel("Error Rate")
plt.title("KNN Classification with Manhattan Distance")
plt.grid(True)
plt.show()


-------------------
## (d).i.B. 
Replace the Euclidean metric with the following metrics5 and test them. Sum- marize the test errors (i.e., when k = k∗) in a table. Use all of your training data and select the best k when {1,6,11,...,196}.

i. Minkowski Distance:

B. with log10(p) ∈ {0.1,0.2,0.3,...,1}. In this case, use the k∗ you found for the Manhattan distance in 1(d)iA. What is the best log10(p)?

In [ ]:
testing_errorsL = []
testing_error_ratesL = []


log10p = list(np.linspace(0.1, 1, 10))
log10p = [round(i, 2) for i in log10p]
# convert the given log10(p) ∈ {0.1,0.2,0.3,...,1} set to one with the p parameter isolated
p_list = [round(10**i, 4) for i in log10p]

for p in p_list:
    
    # We build our model per our k and metric parameter specifications        
    model = knn(n_neighbors=best_k, metric = "minkowski", p=p)
    # We train the model
    model.fit(train_attr_df, train_class_df)

    errors = 0
    # using count to track the indices during the loop and compare the predicted class with the true ones
    count = 0
    # the predict() method classifies the unlabeled data based on the previous training of the model
    for label in model.predict(test_attr_df):
        if label != test_class_df[count]:
            errors += 1
        count += 1
    
    testing_errorsL.append(errors)
    testing_error_rate = errors/len(model.predict(test_attr_df))
    testing_error_ratesL.append(testing_error_rate)

best_testing_error_rate = min(testing_error_ratesL)
best_p = p_list[testing_error_ratesL.index(best_testing_error_rate)]
best_log10p = np.log10(best_p)

# print("log10(p) =", round(best_log10p, 1))
print("For KNN Classification with Minkowski Distance (and k* =", str(best_k) + "):")
print("The best log10(p) =", str(round(best_log10p, 1)) + ", resulting in the best testing error rate of", best_testing_error_rate)

df = pd.DataFrame()
df["k"] = [best_k]*len(p_list)
df["log10(p)"] = log10p
df["p"] = p_list
df["Testing Error Rate"] = testing_error_ratesL
display(df)


# Not asked for, but helpful for interpretation
# Line graph of the testing error rate plotted against log10(p) for log10(p) ∈ {0.1,0.2,0.3,...,1} with Minkowski Distance
plt.plot(log10p, testing_error_ratesL)
plt.xlabel("log10(p)")
plt.ylabel("Testing Error Rate")
plt.title("KNN Classification at k=1 with Minkowski Distance and Varying p")
plt.grid(True)
plt.show()

---------------
## (d).i.C. 
Replace the Euclidean metric with the following metrics and test them. Sum- marize the test errors (i.e., when k = k∗) in a table. Use all of your training data and select the best k when {1,6,11,...,196}.

i. Minkowski Distance:

C. which becomes Chebyshev Distance with p → ∞

In [ ]:
# almost identical to A; could make more efficient using a function

testing_errorsL = []
testing_error_ratesL = []


for k in k_vals:
    
    # We build our model per our k and metric parameter specifications        
    model = knn(n_neighbors=k, metric = "chebyshev")
    # We train the model
    model.fit(train_attr_df, train_class_df)

    errors = 0
    # using count to track the indices during the loop and compare the predicted class with the true ones
    count = 0
    # the predict() method classifies the unlabeled data based on the previous training of the model
    for label in model.predict(test_attr_df):
        if label != test_class_df[count]:
            errors += 1
        count += 1
    
    testing_errorsL.append(errors)
    testing_error_rate = errors/len(model.predict(test_attr_df))
    testing_error_ratesL.append(testing_error_rate)

# initializing best_k such that the while loop starts
best_k = 1
# Was told that 1 cannot be a valid k* since its using itself as a neighbor; need to exclude the possibility k* = 1
while best_k == 1:
    # Even if there are multiple testing error rates of the same best size, since k_vals is sorted least to greatest we still get the optimal k by selecting the first applicable k
    best_testing_error_rate = min(testing_error_ratesL)
    best_k = k_vals[testing_error_ratesL.index(best_testing_error_rate)]
    if 1 in k_vals: k_vals.remove(1)
# need to return the 1, since k_vals is often reused without being redefined
k_vals.insert(0, 1)

print("For KNN Classification with Chebyshev Distance:")
print("k* =", best_k)
print("Resulting in best testing error rate of", best_testing_error_rate)

df = pd.DataFrame()
df["k"] = k_vals
df["Testing Error"] = testing_errorsL
df["Testing Error Rate"] = testing_error_ratesL
display(df)

# Not asked for, but helpful for interpretation
# Line graph of the testing error rate in terms of k for k ∈ {1,6,11,...,196} with Chebyshev Distance
plt.plot(k_vals, testing_error_ratesL)
plt.xlabel("k")
plt.ylabel("Testing Error Rate")
plt.title("KNN Classification with Chebyshev Distance")
plt.grid(True)
plt.show()

-------------
## (d).ii.
Replace the Euclidean metric with the following metrics5 and test them. Sum- marize the test errors (i.e., when k = k∗) in a table. Use all of your training data and select the best k when {1,6,11,...,196}.

ii. Mahalanobis Distance.

In [ ]:
# code to ignore all future warnings
simplefilter(action="ignore", category=FutureWarning)
# action = "default" to return to normal
# Or else, the following code is printing along with the output for every k interation, thus clutering the output area
# /Applications/anaconda3/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1460: FutureWarning: from version 0.25, pairwise_distances for metric='mahalanobis' will require VI to be specified if Y is passed. warnings.warn("from version 0.25, pairwise_distances for "

testing_errorsL = []
testing_error_ratesL = []


for k in k_vals:
    
    # We need the covariance matrix of the test set's features as a paramter for the Mahalanobis metric 
    cov_mat = np.cov(train_attr_df, rowvar = False)
    # if rowvar = True, the rows are treated as the variables
    
    # We build our model per our k and metric parameter specifications
    model = knn(n_neighbors=k, metric = "mahalanobis", metric_params={'V': cov_mat})
    # We train the model
    model.fit(train_attr_df, train_class_df)

    errors = 0
    # using count to track the indices during the loop and compare the predicted class with the true ones
    count = 0
    # the predict() method classifies the unlabeled data based on the previous training of the model
    for label in model.predict(test_attr_df):
        if label != test_class_df[count]:
            errors += 1
        count += 1
    
    testing_errorsL.append(errors)
    testing_error_rate = errors/len(model.predict(test_attr_df))
    testing_error_ratesL.append(testing_error_rate)

# initializing best_k such that the while loop starts
best_k = 1
# Was told that 1 cannot be a valid k* since its using itself as a neighbor; need to exclude the possibility k* = 1
while best_k == 1:
    # Even if there are multiple testing error rates of the same best size, since k_vals is sorted least to greatest we still get the optimal k by selecting the first applicable k
    best_testing_error_rate = min(testing_error_ratesL)
    best_k = k_vals[testing_error_ratesL.index(best_testing_error_rate)]
    if 1 in k_vals: k_vals.remove(1)
# need to return the 1, since k_vals is often reused without being redefined
k_vals.insert(0, 1)

print("For KNN Classification with Mahalanobis Distance:")
print("k* =", best_k)
print("Resulting in best testing error rate of", best_testing_error_rate)

df = pd.DataFrame()
df["k"] = k_vals
df["Testing Error"] = testing_errorsL
df["Testing Error Rate"] = testing_error_ratesL
display(df)

# Not asked for, but helpful for interpretation
# Line graph of the testing error rate in terms of k for k ∈ {1,6,11,...,196} with Mahalanobis Distance
plt.plot(k_vals, testing_error_ratesL)
plt.xlabel("k")
plt.ylabel("Testing Error Rate")
plt.title("KNN Classification with Mahalanobis Distance")
plt.grid(True)
plt.show()

### Summary Table of (d) 

In [ ]:
index = ["k*", "Best Test Error Rate"]
manhattan = [6, 0.11]
minkowski = [6, 0.06]
chebyshev = [16, 0.08]
mahalanobis = [6, 0.15]

df = pd.DataFrame()
df[""] = index
df["Manhattan"] = manhattan
df["Minkowski"] = minkowski
df["Chebyshev"] = chebyshev
df["Mahalanobis"] = mahalanobis
display(df)

------------------
## (e)
The majority polling decision can be replaced by weighted decision, in which the weight of each point in voting is inversely proportional to its distance from the query/test data point. In this case, closer neighbors of a query point will have a greater influence than neighbors which are further away. Use weighted voting with Euclidean, Manhattan, and Chebyshev distances and report the best test errors when k ∈ {1,6,11,16,...,196}.

In [ ]:
# We basically rerun the classification model from the previous question, using "weighted decision"
# A lot of the code is the same, and can likely be optimized by creating a function at the start

dist_typesL = ["Euclidean", "Manhattan", "Chebyshev"]

for dist_type in dist_typesL:

    testing_errorsL = []
    testing_error_ratesL = []
    
    for k in k_vals:

        # Setting the parameter weight="distance" makes the classifier into a wieghted decision one
        # We build our model per our k and metric parameter specifications        
        model = knn(n_neighbors=k, weights="distance", metric = dist_type.lower())
        # We train the model
        model.fit(train_attr_df, train_class_df)

        errors = 0
        # using count to track the indices during the loop and compare the predicted class with the true ones
        count = 0
        # the predict() method classifies the unlabeled data based on the previous training of the model
        for label in model.predict(test_attr_df):
            if label != test_class_df[count]:
                errors += 1
            count += 1

        testing_errorsL.append(errors)
        testing_error_rate = errors/len(model.predict(test_attr_df))
        testing_error_ratesL.append(testing_error_rate)

    # initializing best_k such that the while loop starts
    best_k = 1
    # Was told that 1 cannot be a valid k* since its using itself as a neighbor; need to exclude the possibility k* = 1
    while best_k == 1:
        # Even if there are multiple testing error rates of the same best size, since k_vals is sorted least to greatest we still get the optimal k by selecting the first applicable k
        best_testing_error_rate = min(testing_error_ratesL)
        best_k = k_vals[testing_error_ratesL.index(best_testing_error_rate)]
        if 1 in k_vals: k_vals.remove(1)
    # need to return the 1, since k_vals is often reused without being redefined
    k_vals.insert(0, 1)
    
    print("\nFor KNN Classification by weighted decision with", dist_type, "Distance:")
    print("k* =", best_k)
    print("Resulting in best testing error rate of", best_testing_error_rate)

    df = pd.DataFrame()
    df["k"] = k_vals
    df["Testing Error"] = testing_errorsL
    df["Testing Error Rate"] = testing_error_ratesL
    display(df)

    # Not asked for, but helpful for interpretation
    # Line graph of the testing error rate in terms of k for k ∈ {1,6,11,...,196} based on weighted decision
    plt.plot(k_vals, testing_error_ratesL)
    plt.xlabel("k")
    plt.ylabel("Testing Error Rate")
    plt.title("KNN Classification with %s Distance" %dist_type)
    plt.grid(True)
    plt.show()

### Summary Table of (e) 

In [ ]:
index = ["k*", "Best Test Error Rate"]
euclidean = [6, 0.1]
manhattan = [26, 0.1]
chebyshev = [16, 0.11]

df = pd.DataFrame()
df[""] = index
df["Euclidean"] = euclidean
df["Manhattan"] = manhattan
df["Chebyshev"] = chebyshev
display(df)

---------------
## (f)
What is the lowest training error rate you achieved in this homework?

The lowest training error rate we achieved is 0.0 at 𝑘 = 1 -- in part (c) -- using the Eucidean metric. The KNN training error at k = 1 is based on classifying a sample from the training set by using only itself as its nearest neighbor. So obviously it would have 0 errors.